In [1]:
# load in packages required for analysis.

%matplotlib qt
import os.path

from refnx.dataset import ReflectDataset
from refnx.analysis import Transform, CurveFitter, Objective, GlobalObjective, Parameter
from refnx.reflect import SLD, ReflectModel, MixedReflectModel, MixedSlab, Slab
from MixedLayerSlab import MixedMagSlabs2 #load in custom model for magnetic layers in non-polarised instrument.
import refnx

import dynesty

import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#print what versions we are using.

print('refnx: %s\nscipy: %s\nnumpy: %s' % (refnx.version.version, scipy.version.version, np.version.version))

refnx: 0.1.18
scipy: 1.5.2
numpy: 1.19.2


In [3]:
#load data from .txt files - this can be done smarter but works as a test.

file_path_GMOddod25 = '61167_69.txt'
file_path_GMOCMdod25 = '61203_05.txt'
file_path_GMOhdod25 = '61215_17.txt'

#Define datasets with data loaded

P1_GMO_ddod_25 = ReflectDataset(file_path_GMOddod25)
P1_GMO_CMdod_25 = ReflectDataset(file_path_GMOCMdod25)
P1_GMO_hdod_25 = ReflectDataset(file_path_GMOhdod25)

In [4]:
#now define some SLDs we will use...

#LAYERS#
Si = SLD(2.07, name='Si')
SiO2 = SLD(3.47, name='SiO2')
Fe = SLD(8.02, name='Fe')
FeOx = SLD(7.0, name='FeOx')
GMO_25 = SLD(0.21, name='GMO_25')

#SOLVENTS#
hdod_SLD_1 = SLD(-0.462, name='hdod1')
ddod_par = Parameter(6.7, name="ddodSLDpar", vary=True, bounds=(5, 6.7))
ddod_SLD_2 = SLD(ddod_par, name='ddod2')
CMdod_25_val = 0.352 * ddod_par + (1-0.352) * hdod_SLD_1.real
CMdod_25_SLD = SLD(CMdod_25_val, name='CMdod_25')

In [5]:
"""
Here we define the material layers at the interface.
"""

#Create slabs for model

##SiO2##

SiO2_lay = SiO2(15, 3)

SiO2_lay.thick.setp(vary=True, bounds=(1, 25))

##Fe##
# share the same Fe thickness, SiO2-Fe roughness - we specify variation of parameters here.
SiO2_Fe_R = Parameter(4, 'SiO2_Fe_R', vary=True, bounds=(1, 10))
Fe.real.setp(vary=True, bounds=(7.5, 8.1))
Fe_thick = Parameter(190, 'Fe_t', vary=True, bounds=(170, 210))
Fe_magmom = Parameter(2.1, 'Fe_magmom', vary=True, bounds=(1.9, 2.2))
Fe_ScattLen = Parameter(0.0000945, 'Fe_SL', vary=False)

FeUp_lay = MixedMagSlabs2.USM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)
FeDown_lay = MixedMagSlabs2.DSM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)

##FeOx##
Fe_FeOx_R = Parameter(5, 'FeOx_Fe_R', vary=True, bounds=(1, 15))
FeOx.real.setp(vary=True, bounds=(5.0, 7.2))
FeOx_thick = Parameter(30, 'FeOx_t', vary=True, bounds=(20, 40))
FeOx_m = Parameter(0.5, 'FeOx_m', vary=True, bounds=(0, 1.3))

FeOxUp_lay = MixedMagSlabs2.USM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)
FeOxDown_lay = MixedMagSlabs2.DSM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)

# INTERFACIAL LAYERS#

# ADV #
# share the same FeOx_R across all interfacial layers.
FeOx_R = Parameter(1, 'FeOx_R', vary=True, bounds=(1, 15))

# GMO_25 #
GMO_25_t = Parameter(20, 'GMO_25_t', vary=True, bounds=(10, 30))
GMO_25_lay = GMO_25(GMO_25_t, FeOx_R)
GMO_25_lay.vfsolv.setp(vary=True, bounds=(0, 1))

###INTF-SOLV ROUGHNESS###

# share the Adv-solv roughness.
GMO_25_R = Parameter(1, 'GMO_25_R', vary=True, bounds=(1, 15))

### SOLVENT PARAMS FOR ADV_LAYER SOLVENT
#setup the solvent layers

hdod_GMO25_lay = hdod_SLD_1(0, GMO_25_R)
ddod_GMO25_lay = ddod_SLD_2(0, GMO_25_R)
CMdod_GMO25_lay = CMdod_25_SLD(0, GMO_25_R)

In [6]:
P1_GMO25ddod_U = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | GMO_25_lay | ddod_GMO25_lay
P1_GMO25ddod_D = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | GMO_25_lay | ddod_GMO25_lay

P1_GMO25CMdod_U = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | GMO_25_lay | CMdod_GMO25_lay
P1_GMO25CMdod_D = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | GMO_25_lay | CMdod_GMO25_lay

P1_GMO25hdod_U = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | GMO_25_lay | hdod_GMO25_lay
P1_GMO25hdod_D = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | GMO_25_lay | hdod_GMO25_lay

In [7]:
#now define the models

#these are the scale and background parameters
#intensities
P1GMOddod25_int = Parameter(0.5, 'P1GMOddod25_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1GMOCMdod25_int = Parameter(0.5, 'P1GMOCMdod25_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1GMOhdod25_int = Parameter(0.5, 'P1GMOhdod25_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
#background
P1GMOddod25_bkg = Parameter(1e-6, 'P1GMOddod25_bkg', vary=True, bounds=(1e-7, 2e-5))
P1GMOCMdod25_bkg = Parameter(1e-6, 'P1GMOCMdod25_bkg', vary=True, bounds=(1e-7, 2e-5))
P1GMOhdod25_bkg = Parameter(1e-6, 'P1GMOhdod25_bkg', vary=True, bounds=(1e-7, 2e-5))

#define adv_layer interface objective & global objective. These adv layers have their own SLD.
#models
P1GMOddod25_M = MixedReflectModel((P1_GMO25ddod_U, P1_GMO25ddod_D), scales=(P1GMOddod25_int, P1GMOddod25_int), bkg=P1GMOddod25_bkg, dq=4.7096)
P1GMOCMdod25_M = MixedReflectModel((P1_GMO25CMdod_U, P1_GMO25CMdod_D), scales=(P1GMOCMdod25_int, P1GMOCMdod25_int), bkg=P1GMOCMdod25_bkg, dq=4.7096)
P1GMOhdod25_M = MixedReflectModel((P1_GMO25hdod_U, P1_GMO25hdod_D), scales=(P1GMOhdod25_int, P1GMOhdod25_int), bkg=P1GMOhdod25_bkg, dq=4.7096)
#objectives
P1GMOddod25_obj = Objective(P1GMOddod25_M, P1_GMO_ddod_25, transform=Transform('logY'))
P1GMOCMdod25_obj = Objective(P1GMOCMdod25_M, P1_GMO_CMdod_25, transform=Transform('logY'))
P1GMOhdod25_obj = Objective(P1GMOhdod25_M, P1_GMO_hdod_25, transform=Transform('logY'))
#global obj
glob_objective = GlobalObjective([P1GMOddod25_obj, P1GMOCMdod25_obj, P1GMOhdod25_obj])

In [8]:
### BE SENSIBLE AND HAVE A LOOK AT YOUR DATA BEFORE NS ###

# P1hdod_J_adv_obj.plot()
# P1ddod_J_adv_obj.plot()

### ALSO HAVE A LOOK AT THE SLD TO CHECK ALL LOOKS FINE ###

plt.plot(*P1_GMO25CMdod_U.sld_profile())
plt.plot(*P1_GMO25CMdod_D.sld_profile())
plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
plt.xlabel('distance / $\AA$');

### makes sense to look at the list of varying parameters to ensure no mistakes ###
#print(glob_objective.varying_parameters().names())
print(glob_objective)

_______________________________________________________________________________

--Global Objective--
________________________________________________________________________________
Objective - 1942508148040
Dataset = 61167_69
datapoints = 185
chi2 = 10030.078995771571
Weighted = True
Transform = Transform('logY')
________________________________________________________________________________
Parameters:       ''       
[Parameters(data=[Parameters(data=[Parameter(value=0.5, name='P1GMOddod25_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None), Parameter(value=0.5, name='P1GMOddod25_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None)], name='scale factors'), Parameter(value=1e-06, name='P1GMOddod25_bkg', vary=True, bounds=Interval(lb=1e-07, ub=2e-05), constraint=None), Parameter(value=4.7096, name='dq - resolution', vary=False, bounds=Interval(lb=-np.inf, ub=np.inf), constraint=None), Parameter(value=0.0, name='q_offset', vary=False, bounds=Interval(lb=-

In [12]:
### Nested Sampling ### Now we do the nested sampling!
nested_sampler = dynesty.NestedSampler(glob_objective.logl, glob_objective.prior_transform, ndim=len(glob_objective.varying_parameters()), nlive=1500)
nested_sampler.run_nested()

90456it [10:02:52,  2.50it/s, +1500 | bound: 564 | nc: 1 | ncall: 2180039 | eff(%):  4.218 | loglstar:   -inf < 1245.509 <    inf | logz: 1184.127 +/-  0.335 | dlogz:  0.002 >  1.509]


In [13]:
ns_results_sep_adv = nested_sampler.results
#print(ns_results_sep_adv)

from dynesty import plotting as dyplot

# Plot a summary of the run.
#rfig, raxes = dyplot.runplot(ns_results_sep_adv, logplot=True) #this doesn't seem to like evidences that are large.
#seems the plotting function plots evidence in a linear fashion, and hence tries to calculate e^(ln(evidence)).
#if ln(evidence) is large, i.e 800, then we get an overflow issue as this number is HUGE.

# Plot traces and 1-D marginalized posteriors.
#tfig, taxes = dyplot.traceplot(ns_results_sep_adv)

#Plot the 2-D marginalized posteriors.
# labelz = glob_objective.varying_parameters().names()
# print(labelz[2:10])
# cfig, caxes = dyplot.cornerplot(ns_results_sep_adv, dims=[2, 3, 4, 5, 6, 7], labels=labelz)

print(ns_results_sep_adv.logz[-1], ns_results_sep_adv.logzerr[-1])

1184.126785103523 0.3347730054213903
